In [1]:
# Common imports
import numpy as np, matplotlib.pyplot as plt, pandas as pd
import os

# For output stability across multiple runs of the notebook
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

In [36]:
import csv

# Might have stolen from stackoverflow
def decomment(file):
    for line in file:
        raw = line.split("#")[0].strip()
        if raw: yield raw

fname="data/ADNI_131_S_0123_20151214162609918/stats/fixedlh.stats"
datafile="data/ADNI_131_S_0123_20151214162609918/stats/lh.aparc.a2009s.stats"

BS = 0 # Brain Structure Name
SA = 2 # Surface Area (mm^2)
TH = 4 # Average Thickness

with open(datafile) as f:
    data = []
    for row in decomment(f):
        stats = row.split()
        data.append([stats[i] for i in [BS,SA,TH]])
    


In [64]:
data = np.array(data)
columns = [structure+"_SA" for structure in data[:,0]]
columns += [structure+"_TH" for structure in data[:,0]]

raw_data = [data[:,1:3].flatten(order="F")] # Column-wise collapse 2D array
raw_data

[array(['889', '992', '736', '1116', '405', '1242', '741', '657', '324',
        '212', '1130', '850', '148', '543', '2214', '3766', '214', '294',
        '1218', '986', '1468', '1934', '903', '1593', '1165', '1455',
        '1797', '1136', '1685', '1495', '621', '187', '272', '1243', '252',
        '621', '1409', '1467', '183', '73', '677', '1149', '724', '1498',
        '2243', '701', '254', '863', '1134', '392', '293', '1350', '862',
        '1664', '271', '1846', '762', '924', '639', '631', '1386', '208',
        '606', '705', '1393', '763', '1721', '1350', '728', '328', '612',
        '539', '3429', '215', '2.044', '2.136', '1.914', '2.599', '2.477',
        '2.537', '2.460', '2.359', '2.640', '1.901', '1.583', '2.490',
        '2.499', '2.463', '2.196', '2.377', '3.070', '3.622', '2.344',
        '1.778', '2.508', '1.716', '2.429', '2.474', '2.150', '2.427',
        '2.006', '1.854', '2.388', '2.285', '2.433', '1.648', '2.566',
        '2.872', '2.275', '2.370', '2.606', '2.638',

In [66]:
df = pd.DataFrame(raw_data, columns=columns)
df

,G_and_S_frontomargin_SA,G_and_S_occipital_inf_SA,G_and_S_paracentral_SA,G_and_S_subcentral_SA,G_and_S_transv_frontopol_SA,G_and_S_cingul-Ant_SA,G_and_S_cingul-Mid-Ant_SA,G_and_S_cingul-Mid-Post_SA,G_cingul-Post-dorsal_SA,G_cingul-Post-ventral_SA,...,S_parieto_occipital_TH,S_pericallosal_TH,S_postcentral_TH,S_precentral-inf-part_TH,S_precentral-sup-part_TH,S_suborbital_TH,S_subparietal_TH,S_temporal_inf_TH,S_temporal_sup_TH,S_temporal_transverse_TH
0,889,992,736,1116,405,1242,741,657,324,212,...,1.761,2.365,1.730,2.086,1.658,1.980,1.905,2.254,2.169,1.990
